In [1]:
import agentpy as ap
import matplotlib.pyplot as plt
import IPython
import random
import itertools
import numpy as np
import json
import requests

In [2]:
# Type of move 
LEFT_RIGHT_MOVE = 1
RIGHT_LEFT_MOVE = 2
UP_DOWN_MOVE = 3
DOWN_UP_MOVE = 4

VALID_MOVES = {
    LEFT_RIGHT_MOVE: (0, 1),
    RIGHT_LEFT_MOVE: (0, -1),
    UP_DOWN_MOVE: (1,0),
    DOWN_UP_MOVE: (-1,0)
}


# Colors
CAR_COLOR = 1
YELLOW_COLOR = 101
GREEN_COLOR = 102
RED_COLOR = 103
GRAY_COLOR = 104

ID = 0

url = "http://localhost:8080/"

jsonObject = {}

In [3]:
class CarAgent(ap.Agent):

    current_move_type = 0
    carsObj = []
    def setup(self):
        # return value of the type of move
        self.current_move_type = random.choice(list(VALID_MOVES.keys()))

        # Atributes for the JSON
        self.id = 0
        self.x=0
        self.y=0
        self.z=0
        
        self.speed = 0.2
        self.crossed = False
        self.num = 0
        self.num2 = 0

        global flag
        global flag2
        global flag3
        flag = 0
        flag2 = 0
        flag3 = 0

        self.res_y = {key: VALID_MOVES[key] for key in VALID_MOVES.keys() & {UP_DOWN_MOVE, RIGHT_LEFT_MOVE}}
        self.nextMove_y = random.choice(list(self.res_y.keys()))
        
        self.res_y_2 = {key: VALID_MOVES[key] for key in VALID_MOVES.keys() & {DOWN_UP_MOVE, LEFT_RIGHT_MOVE}}
        self.nextMove_y_2 = random.choice(list(self.res_y_2.keys()))
        
        self.res_x = {key: VALID_MOVES[key] for key in VALID_MOVES.keys() & {RIGHT_LEFT_MOVE, DOWN_UP_MOVE}}
        self.nextMove_x = random.choice(list(self.res_x.keys()))
        
        self.res_x_2 = {key: VALID_MOVES[key] for key in VALID_MOVES.keys() & {LEFT_RIGHT_MOVE, UP_DOWN_MOVE}}
        self.nextMove_x_2 = random.choice(list(self.res_x_2.keys()))
        
    def get_move(self):
        # Allows cars to randomize speed 
        if self.y != 6 and self.y != -5 and self.x != 5 and self.x != -4 and self.speed > 0.1:
            # Returns the coordinate of the choosen move
            return VALID_MOVES[self.current_move_type]
        return(0,0)
    
    def get_move_2(self):     
        if flag2 == 1:
            return(0,0)
        return(0,-1)
    
    def get_move_3(self):
        if flag3 == 1:
            return(0,0)
        return(0,1)
        
    def continue_path_y_1(self):
        if self.nextMove_y == 2:
            self.num += 1
            if flag == 1:
                self.num = 0
            if self.num >= 9:
                return(0,0)
        return self.res_y[self.nextMove_y]
    
    def continue_path_y_2(self):
        if self.nextMove_y_2 == 1:
            self.num += 1
            if flag == 1:
                self.num = 0
                #self.flag = 0
            if self.num >= 9:
                return(0,0)
        return self.res_y_2[self.nextMove_y_2]
    
    def continue_path_x_1(self):
        if self.nextMove_x == 4:
            #print(self.num2)
            self.num2 += 1
            if self.num2 >= 9:
                return(0,0)
        return self.res_x[self.nextMove_x]
    
    def continue_path_x_2(self):
        if self.nextMove_x_2 == 3:
            #print(self.num2)
            self.num2 += 1
            if self.num2 >= 9:
                return(0,0)
        return self.res_x_2[self.nextMove_x_2]

    def send_requests(self, json_string):
        try:
            requests.post(url, json_string)
        except:
            print("Unable to send positions")
        #print(self.current_car_positions)

    def get_coords(self, coordinates_move):
        if(coordinates_move==(0, 1)):
            self.x = self.x+1
        elif(coordinates_move==(0, -1)):
            self.x = self.x-1
        if(coordinates_move==(1,0)):
            self.y = self.y+1
        elif(coordinates_move==(-1, 0)):
            self.y = self.y-1


        #self.carsObj({'x': self.x, 'y': self.y})

        #self.carsObj['current_car_positions'][iterator]['y'] = self.y

        #iterator +=1
        #json_string = json.dumps(self.carsObj)
        #print(self.carsObj)
        #self.send_requests(json_string)


In [4]:
class LightAgent(ap.Agent):
    def setup(self):

        # Iitilize the default color
        self.current_color = GREEN_COLOR
        
        self.step_time = 0.1         # Tiempo que dura cada paso de la simulación
        
        # Extra colors
        self.green_time = 10
        self.red_time = 8
        self.yellow_time = 3

    def update(self):
        self.state_time += self.step_time

        if self.state == 0:
            # Caso en el que el semáforo está en verde
            if self.state_time >= self.green_time:
                self.state = 1
                self.current_color = YELLOW_COLOR
                self.state_time = 0
        elif self.state == 1:
            # Caso en el que el semáforo está en amarillo
            if self.state_time >= self.yellow_time:
                self.state = 2
                self.current_color = RED_COLOR
                self.state_time = 0
        elif self.state == 2:
            # Caso en el que el semáforo está en rojo
            if self.state_time >= self.red_time:
                self.state = 0
                self.current_color = GREEN_COLOR
                self.state_time = 0

    def set_green(self):     
        self.state = 0
        self.current_color = GREEN_COLOR
        self.state_time = 0

    def set_yellow(self):     
        self.state = 1
        self.current_color = YELLOW_COLOR
        self.state_time = 0

    def set_red(self):     
        self.state = 2
        self.current_color = RED_COLOR
        self.state_time = 0

In [5]:
class SidewalkAgent(ap.Agent):
    def setup(self):
        # Iitilize the default color
        self.current_color = GRAY_COLOR

In [6]:
{
    'cars': 
        [
            {'id': 0, 'x': 0, 'y': 0},
            {'id': 1, 'x': 0, 'y': 0},
            {'id': 2, 'x': 0, 'y': 0},
            {'id': 3, 'x': 0, 'y': 0},
            {'id': 4, 'x': 0, 'y': 0},
            {'id': 5, 'x': 0, 'y': 0},
            {'id': 6, 'x': 0, 'y': 0},
            {'id': 7, 'x': 0, 'y': 0}, 
            {'id': 8, 'x': 0, 'y': 0}, 
            {'id': 9, 'x': 0, 'y': 0}, 
            {'id': 10, 'x': 0, 'y': 0}, 
            {'id': 11, 'x': 0, 'y': 0}, 
            {'id': 12, 'x': 0, 'y': 0}, 
            {'id': 13, 'x': 0, 'y': 0}, 
            {'id': 14, 'x': 0, 'y': 0}
        ], 
    'lights': 
        [
            {'id': 0, 'state': 'YELLOW_COLOR'}, 
            {'id': 1, 'state': 'YELLOW_COLOR'}, 
            {'id': 2, 'state': 'YELLOW_COLOR'}, 
            {'id': 3, 'state': 'YELLOW_COLOR'}
        ]
}

{'cars': [{'id': 0, 'x': 0, 'y': 0},
  {'id': 1, 'x': 0, 'y': 0},
  {'id': 2, 'x': 0, 'y': 0},
  {'id': 3, 'x': 0, 'y': 0},
  {'id': 4, 'x': 0, 'y': 0},
  {'id': 5, 'x': 0, 'y': 0},
  {'id': 6, 'x': 0, 'y': 0},
  {'id': 7, 'x': 0, 'y': 0},
  {'id': 8, 'x': 0, 'y': 0},
  {'id': 9, 'x': 0, 'y': 0},
  {'id': 10, 'x': 0, 'y': 0},
  {'id': 11, 'x': 0, 'y': 0},
  {'id': 12, 'x': 0, 'y': 0},
  {'id': 13, 'x': 0, 'y': 0},
  {'id': 14, 'x': 0, 'y': 0}],
 'lights': [{'id': 0, 'state': 'YELLOW_COLOR'},
  {'id': 1, 'state': 'YELLOW_COLOR'},
  {'id': 2, 'state': 'YELLOW_COLOR'},
  {'id': 3, 'state': 'YELLOW_COLOR'}]}

In [7]:
class StreetModel(ap.Model):
    def setup(self):
        global iterator
        global initial_car_location
        iterator = 0
        
        #Create first the X lights
        x_separation = self.p.lights_separation["X"]
        y_separation = self.p.lights_separation["Y"]

        x_valid = range(x_separation, self.p.size["X"], x_separation)
        y_valid = range(y_separation, self.p.size["Y"], y_separation)

        lights_location = list(itertools.product(y_valid, x_valid))
        n_lights = len(lights_location)

        self.lights = ap.AgentList(self, n_lights, LightAgent)
        self.lights.is_car = False
        self.lights.type_agent = YELLOW_COLOR
        
        self.lights.step_time =  self.p.step_time
        self.lights.green_time = self.p.green
        self.lights.yellow_time = self.p.yellow
        self.lights.red_time = self.p.red

        self.cars = ap.AgentList(self, int(self.p["cars"]), CarAgent)

        self.lightsObj = []

        for light in self.lights:
            light.id = iterator
            iterator += 1
            lightObject = {
                "id": light.id,
                "state": 'YELLOW_COLOR'
            }
            self.lightsObj.append(lightObject)

        iterator = 0
        self.carsObj = []
        for car in self.cars:
            car.id = iterator
            iterator += 1
            carObject = {
                    "id": car.id,
                    "x":car.x,
                    "y":car.y,
                    "z":car.z
                }
            self.carsObj.append(carObject)

        self.cars.is_car = True
        self.cars.type_agent = CAR_COLOR

        jsonObject["cars"] = self.carsObj
        jsonObject["lights"] = self.lightsObj

        #print(jsonObject)
        #create grid (street)
        self.street = ap.Grid(self, [self.p["size"]["Y"], self.p["size"]["X"]], track_empty=True, torus=True)
        
        n_sidewalks = 5
        self.sidewalk = ap.AgentList(self, n_sidewalks, SidewalkAgent)
        self.sidewalk.is_car = False
        self.sidewalk.type_agent = GRAY_COLOR
        
        # Add sidewalks
        for a in range(5):
            for b in range(5):
                s_list = []
                sidewalk_location_1 = a, b
                s_list.append(sidewalk_location_1)
                if s_list[0] == lights_location[0]:
                    s_list.pop(0)
                self.street.add_agents(self.sidewalk, s_list)
                
        for a in range(5):
            for b in range(4):
                s_list = []
                sidewalk_location_1 = a, b + 8
                s_list.append(sidewalk_location_1)
                if s_list[0] == lights_location[1]:
                    s_list.pop(0)
                self.street.add_agents(self.sidewalk, s_list)
                
        for a in range(4):
            for b in range(5):
                s_list = []
                sidewalk_location_1 = a + 8, b
                s_list.append(sidewalk_location_1)
                if s_list[0] == lights_location[2]:
                    s_list.pop(0)
                self.street.add_agents(self.sidewalk, s_list)
        
        for a in range(4):
            for b in range(4):
                s_list = []
                sidewalk_location_1 = a + 8, b + 8
                s_list.append(sidewalk_location_1)
                if s_list[0] == lights_location[3]:
                    s_list.pop(0)
                self.street.add_agents(self.sidewalk, s_list)
        
        #add the lights agents
        self.street.add_agents(self.lights, lights_location)
        
        # Cars locations
        valid_car_location = [(7, 0), (11, 7), (5, 11), (0, 5)]
        
        initial_car_location = []
        for car in self.cars:
            position = random.choice(valid_car_location)
            initial_car_location += [position]
            car.x = position[1]
            car.y = position[0]

        for car, initial_location in zip(self.cars, initial_car_location):
            
            if initial_location[0] == 7:
                #up
                car.current_move_type = LEFT_RIGHT_MOVE
            elif initial_location[0] == 5:
                #down
                car.current_move_type = RIGHT_LEFT_MOVE
            elif initial_location[0] == 11:
                #left
                car.current_move_type = DOWN_UP_MOVE
            elif initial_location[0] == 0:
                #right
                car.current_move_type = UP_DOWN_MOVE
        
        self.street.add_agents(self.cars, initial_car_location)

        
    def step(self):
        iterator = 0
        global initial_car_location
        
        for car in self.cars:
            coordinates_move = car.get_move()
            
            if(coordinates_move==(0, 1)):
                car.x = car.x+1
            elif(coordinates_move==(0, -1)):
                car.x = car.x-1
            if(coordinates_move==(1,0)):
                car.y = car.y+1
            elif(coordinates_move==(-1, 0)):
                car.y = car.y-1
                
            self.carsObj[iterator]['x'] = car.x
            self.carsObj[iterator]['y'] = car.y

            iterator +=1

        jsonObjec = {}
        jsonObjec["cars"] = self.carsObj
        jsonObjec["lights"] = self.lightsObj
        json_string_object = json.dumps(jsonObjec)

        try:
            requests.post(url, json_string_object)
        except:
            print("Unable to send positions")


        #json_string = json.dumps(self.carsObj)
        #print(json_string)
      

        if self.p.step_time <= self.p.green:
            self.lights[0].type_agent = GREEN_COLOR
            self.lights[1].type_agent = RED_COLOR
            self.lights[2].type_agent = RED_COLOR
            self.lights[3].type_agent = GREEN_COLOR

            for ctr in range(len(initial_car_location)):
                if initial_car_location[ctr][0] == 0:
                    coordinates_move = self.cars[ctr].get_move()
                    self.cars[ctr].get_coords(coordinates_move)
                    self.street.move_by(self.cars[ctr], coordinates_move)
                    if coordinates_move == (0,0):
                        next_move = self.cars[ctr].continue_path_y_1()
                        self.cars[ctr].get_coords(next_move)
                        self.street.move_by(self.cars[ctr], next_move)
                elif initial_car_location[ctr][0] == 11:
                    coordinates_move = self.cars[ctr].get_move()
                    self.cars[ctr].get_coords(coordinates_move)
                    self.street.move_by(self.cars[ctr], coordinates_move)
                    if coordinates_move == (0,0):
                        next_move = self.cars[ctr].continue_path_y_2()
                        self.cars[ctr].get_coords(next_move)
                        self.street.move_by(self.cars[ctr], next_move)
                elif initial_car_location[ctr][0] == 5:
                    coordinates_move = self.cars[ctr].get_move()
                    self.cars[ctr].get_coords(coordinates_move)
                    self.street.move_by(self.cars[ctr], coordinates_move)
                elif initial_car_location[ctr][0] == 7:
                    coordinates_move = self.cars[ctr].get_move()
                    self.cars[ctr].get_coords(coordinates_move)
                    self.street.move_by(self.cars[ctr], coordinates_move)
                else:
                    pass
            
        elif self.p.step_time > self.p.green and self.p.step_time <= self.p.yellow:
            speed = 2
            self.lights[0].type_agent = YELLOW_COLOR
            self.lights[1].type_agent = RED_COLOR
            self.lights[2].type_agent = RED_COLOR
            self.lights[3].type_agent = YELLOW_COLOR

            for ctr in range(len(initial_car_location)):
                if initial_car_location[ctr][0] == 0:
                    coordinates_move = self.cars[ctr].get_move()
                    self.cars[ctr].get_coords(coordinates_move)
                    self.street.move_by(self.cars[ctr], coordinates_move)
                    if coordinates_move == (0,0):
                        next_move = self.cars[ctr].continue_path_y_1()
                        self.cars[ctr].get_coords(next_move)
                        self.street.move_by(self.cars[ctr], 
                                            (speed*next_move[0], speed*next_move[1]))
                elif initial_car_location[ctr][0] == 11:
                    coordinates_move = self.cars[ctr].get_move()
                    self.cars[ctr].get_coords(coordinates_move)
                    self.street.move_by(self.cars[ctr], coordinates_move)
                    if coordinates_move == (0,0):
                        next_move = self.cars[ctr].continue_path_y_2()
                        self.cars[ctr].get_coords(next_move)
                        self.street.move_by(self.cars[ctr], 
                                            (speed*next_move[0], speed*next_move[1]))
                elif initial_car_location[ctr][0] == 5:
                    coordinates_move = self.cars[ctr].get_move()
                    self.cars[ctr].get_coords(coordinates_move)
                    self.street.move_by(self.cars[ctr], coordinates_move)
                elif initial_car_location[ctr][0] == 7:
                    coordinates_move = self.cars[ctr].get_move()
                    self.cars[ctr].get_coords(coordinates_move)
                    self.street.move_by(self.cars[ctr], coordinates_move)
                else:
                    pass
            
        else:
            global flag 
            global flag2
            global flag3
            flag = 1
            
            self.lights[0].type_agent = RED_COLOR
            self.lights[1].type_agent = GREEN_COLOR
            self.lights[2].type_agent = GREEN_COLOR
            self.lights[3].type_agent = RED_COLOR

            for ctr in range(len(initial_car_location)):
                if initial_car_location[ctr][0] == 0:
                    next_move = self.cars[ctr].continue_path_y_1()
                    self.cars[ctr].get_coords(next_move)
                    #print(next_move)
                    if next_move[0] == 1:
                        next_move = (0,0)
                        self.cars[ctr].get_coords(next_move)
                    else:
                        self.street.move_by(self.cars[ctr], next_move)
                elif initial_car_location[ctr][0] == 11:
                    next_move = self.cars[ctr].continue_path_y_2()
                    self.cars[ctr].get_coords(next_move)
                    if next_move[0] == -1:
                        next_move = (0,1)
                        self.cars[ctr].get_coords(next_move)
                        self.street.move_by(self.cars[ctr], next_move)
                    else:
                        self.street.move_by(self.cars[ctr], next_move)
                elif initial_car_location[ctr][0] == 5:
                    coordinates_move = self.cars[ctr].get_move_2()
                    self.cars[ctr].get_coords(coordinates_move)
                    self.street.move_by(self.cars[ctr], coordinates_move)
                    #flag2 = 1
                    if coordinates_move == (0,0):
                        next_move = self.cars[ctr].continue_path_x_1()
                        self.cars[ctr].get_coords(next_move)
                        self.street.move_by(self.cars[ctr], next_move)
                elif initial_car_location[ctr][0] == 7:
                    coordinates_move = self.cars[ctr].get_move_3()
                    self.cars[ctr].get_coords(coordinates_move)
                    self.street.move_by(self.cars[ctr], coordinates_move)
                    #flag3 = 1
                    if coordinates_move == (0,0):
                        next_move = self.cars[ctr].continue_path_x_2()
                        self.cars[ctr].get_coords(next_move)
                        self.street.move_by(self.cars[ctr], next_move)
                else:
                    pass
            
            flag2 = 1
            flag3 = 1
        
        self.p.step_time += 1
        
    def end(self):
        pass

In [8]:
# Define parameters

parameters = {
    'step_time': 0.1,
    'cars': 5,
    'lights_separation': {'X': 4, 'Y':4},
    'size': {'X': 12, 'Y': 12}, # Height and length of the grid
    'green': 80.0,          
    'yellow': 120.0,         
    'red': 200.0,
    'steps': 200,
}

In [9]:

def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('type_agent')
    color_dict = {CAR_COLOR:'#0000FF', YELLOW_COLOR:'#FFFF00', GREEN_COLOR:'#00FF00', RED_COLOR:'#FF0000', GRAY_COLOR: '#A49E9E',None:'#571150'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a forest fire\n"
                 f"Time-step: {model.t}, Trees left: ")
                 #f"{len(model.agents.select(model.agents.condition == 0))}")

fig, ax = plt.subplots(figsize=(6,6))
model = StreetModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

Unable to send positions


In [ ]:
model.reporters